# Information extraction with Anthropic Claude

## Overview

In this example, you are going to ingest text representing description of New York City (using a csv obtained from the train split of the [`rajpurkar/squad`](https://huggingface.co/datasets/rajpurkar/squad) dataset of HuggingFace) directly into Amazon Bedrock API (using Anthropic Claude model) and give it an instruction to extract information from it.


In this notebook:

1. Ingest the csv (containing text, along with questions and ground truth data extracted from it)
1. Use this text as input data for the model
1. The foundation model processes the input data
1. Model returns a response with the data extracted from the ingested text

## Setup

In [ ]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57" \
    "anthropic"

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
import json
import os
import sys

import boto3
import botocore


module_path = "../.."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

## New York City small dataset
It is a dataset of informations about NYC, with associated questions and answers: this allows us to get feedback on the correctness of our extractions.\
We ingest the dataset.

In [ ]:
import pandas as pd
df=pd.read_csv("../data/nyc.csv")

Let's visualize some data from the dataset

In [ ]:
df.head()

### We can leverage the structure of the dataset to test Claude's ability in information extraction

Let's select a subset of the DataFrame rows and for each of them let's use the question field as a prompt. Then, we can compare the answer given by the model with the one available on the DF.

In [ ]:
df_short=df[:10]

To do so, we will have to iterate over the rows and perform a call to the model in each iteration. It's useful to build a function that accepts the question and the text as inputs, performs the request to the model and returns the answer.

In [ ]:
def request_to_model(context,question):
    prompt_data=f"\n\nHuman:Answer the following question using the data read in the text.\nQuestion: {question}\nText:{context}\n\nAssistant:"
    try:

        body = json.dumps({"prompt": prompt_data,"max_tokens_to_sample":1024})
        modelId = "anthropic.claude-v2"
        accept = "application/json"
        contentType = "application/json"

        response = boto3_bedrock.invoke_model(
            body=body, modelId=modelId, accept=accept, contentType=contentType
        )
        response_body = json.loads(response.get("body").read())
        return response_body.get("completion")

    except botocore.exceptions.ClientError as error:

        if error.response['Error']['Code'] == 'AccessDeniedException':
            print(f"\x1b[41m{error.response['Error']['Message']}\
                    \nTo troubeshoot this issue please refer to the following resources.\
                    \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                    \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

        else:
            raise error


Let's execute the following loop to attach the model answers to the shortened DataFrame, in order to confront them easily with the ground truth

In [ ]:
for index, row in df_short.iterrows():
    df_short.loc[index,"llm_answer"]=request_to_model(row["context"],row["question"])

Now, let's look at the results

In [ ]:
display(df_short)

In [ ]:
for i,row in df_short.iterrows():
    print(f"Ground truth: {row['answers']}, LLM output: {row['llm_answer']}")
    print("\n")

We notice that the LLM gives out pretty long answers, what if we would want the most concise answer possible (similar to the ground truth ones)? Let's try modifying the prompt

In [ ]:
def request_to_model_concise(context,question):
    prompt_data=f"\n\nHuman:Answer the following question using the data read in the text.\nQuestion: {question}\nText:{context}\nJust provide the information, without mentioning what you read in the text.\n\nAssistant:"
    try:

        body = json.dumps({"prompt": prompt_data,"max_tokens_to_sample":1024})
        modelId = "anthropic.claude-v2"
        accept = "application/json"
        contentType = "application/json"

        response = boto3_bedrock.invoke_model(
            body=body, modelId=modelId, accept=accept, contentType=contentType
        )
        response_body = json.loads(response.get("body").read())
        return response_body.get("completion")

    except botocore.exceptions.ClientError as error:

        if error.response['Error']['Code'] == 'AccessDeniedException':
            print(f"\x1b[41m{error.response['Error']['Message']}\
                    \nTo troubeshoot this issue please refer to the following resources.\
                    \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                    \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

        else:
            raise error


Let's test again, executing the same loop to get the results

In [ ]:
for index, row in df_short.iterrows():
    df_short.loc[index,"llm_answer"]=request_to_model_concise(row["context"],row["question"])

In [ ]:
display(df_short)

In [ ]:
for i,row in df_short.iterrows():
    print(f"Ground truth: {row['answers']}, LLM output: {row['llm_answer']}")
    print("\n")

As you can see, by slightly changing the prompt we obtained results that are similar to those of the ground truth!

### Now, you try experimenting with Information Extraction! You have many other rows in the NYC csv to test and in the 'data' folder there is also another small dataset with text talking about Buddhism. 
### Try experimenting with different prompts, you could get even better output formats using the data as examples for Few-Shot or CoT prompting! Feel free to use the code we just saw as a base.

#### Exercise - other csv

In [4]:
import pandas as pd
df_2=pd.read_csv("../data/buddhism.csv")
df_2_short=df_2[:10]
display(df_2_short)

,context,question,answers
0,Buddhism /ˈbudɪzəm/ is a nontheistic religion[...,What type of religion is Buddhism?,nontheistic
1,Two major extant branches of Buddhism are gene...,What are the two major branches of Buddhism?,"Theravada (""The School of the Elders"") and Mah..."
2,"In Theravada Buddhism, the ultimate goal is th...",What is the goal of Theravada Buddhism?,the attainment of the sublime state of Nirvana
3,Buddhist schools vary on the exact nature of t...,What are the Three Jewels of the Buddhist trad...,"the Buddha, the Dharma (the teachings), and th..."
4,This narrative draws on the Nidānakathā of the...,The Nidānakathā of the Jataka tales of the The...,Nidānakathā of the Jataka tales of the Theravada
5,"According to author Michael Carrithers, while ...","Who said ""the outline of the life must be true...",Michael Carrithers
6,The evidence of the early texts suggests that ...,"According to early texts, when was Siddhārtha ...",fifth century BCE
7,"According to this narrative, shortly after the...",What was the name of the astrologer who visite...,Asita
8,Śuddhodana was determined to see his son becom...,What did Śuddhodana do to ensure that his son ...,prevented him from leaving the palace grounds
9,Gautama first went to study with famous religi...,What was the first thing Gautama did on his sp...,went to study with famous religious teachers o...
